In [9]:
from census import Census
from api_keys import census_key
import json
import requests
import pandas as pd

c = Census(census_key)

In [10]:
#Estimate!!Total!!Population for whom poverty status is determined
pop_table = 'S1701_C01_001E'
#Estimate!!Below poverty level!!Population for whom poverty status is determined
poverty_table = 'S1701_C02_001E'

subject_table = 'https://api.census.gov/data/2018/acs/acs5/subject?'
get = f'NAME,{pop_table},{poverty_table}'
geo = 'county:*&in=state:17'
url = f'{subject_table}get={get}&for={geo}&key={census_key}'
response = requests.get(url)
response

<Response [200]>

In [11]:
poverty_data = response.json()
poverty_data[:5]

[['NAME', 'S1701_C01_001E', 'S1701_C02_001E', 'state', 'county'],
 ['Jersey County, Illinois', '21109', '1944', '17', '083'],
 ['Putnam County, Illinois', '5686', '501', '17', '155'],
 ['De Witt County, Illinois', '15766', '1739', '17', '039'],
 ['Fayette County, Illinois', '19885', '3551', '17', '051']]

In [12]:
poverty_df = pd.DataFrame(poverty_data[1:], columns=poverty_data[0], dtype='int32')
poverty_df.head()

,NAME,S1701_C01_001E,S1701_C02_001E,state,county
0,"Jersey County, Illinois",21109,1944,17,083
1,"Putnam County, Illinois",5686,501,17,155
2,"De Witt County, Illinois",15766,1739,17,039
3,"Fayette County, Illinois",19885,3551,17,051
4,"Lee County, Illinois",31258,3392,17,103


In [13]:
poverty_series = poverty_df[poverty_table].astype('int32')
pop_series=poverty_df[pop_table].astype('int32')
poverty_df['Poverty Rate'] = poverty_series/pop_series * 100

In [14]:
draft_df = poverty_df.sort_values('Poverty Rate')\
            .rename(columns={poverty_table:'Poverty Population',\
                            pop_table:'Total Population'})
draft_df.head()

,NAME,Total Population,Poverty Population,state,county,Poverty Rate
63,"Monroe County, Illinois",33672,1673,17,133,4.968520
99,"Kendall County, Illinois",124009,6598,17,093,5.320582
67,"Piatt County, Illinois",16335,970,17,147,5.938170
34,"DuPage County, Illinois",918650,62203,17,043,6.771132
18,"Will County, Illinois",678516,47519,17,197,7.003372


In [7]:
draft_df.to_csv('acs5_IL_county_calculatedpovertyrate.csv')